In [300]:
#design of the program
#需求
#训练数据的浓度是可以被更改的（设置成许多的小函数 应该就可以参数一键直接改掉了）
#只要我想输入哪些药物 就能输入哪些 
#我想改component就能直接改
#我想用pca或者lda能够自己选定
#设置的药物label能直接根据我的需求去设定
#能有filter功能，只要我想要 就要给我更改filter的数据
#


In [301]:
import pandas as pd
df=pd.read_csv('drug_with_fields.csv')
df

,drug_name,default,ks1,ks2,ks3,ks4,ks5,ks6,ks7,ks8,...,ks229,ks230,ks231,ks232,ks233,ks234,ks235,ks236,ks237,ks238
0,Gemcitabine+17AAG+1,0,-0.41708,-0.541930,-0.36498,-0.306320,-0.799290,-0.58251,-0.33667,-0.31465,...,0.184100,0.060118,-0.171780,0.213250,0.209031,0.060118,-0.502310,-0.404050,-0.200240,0.226750
1,Gemcitabine+17AAG+1,0,-0.40000,-0.593380,-0.38970,-0.424760,-0.884610,-0.58410,-0.32108,-0.29055,...,0.122220,0.090601,-0.188930,0.172610,0.155130,0.090601,-0.550240,-0.413960,-0.223860,0.212200
2,Gemcitabine+17AAG+2,0,-0.32967,-0.544500,-0.28783,-0.347170,-0.825170,-0.53008,-0.24200,-0.20550,...,-0.104500,-0.123170,-0.069417,-0.168330,-0.156250,-0.123170,-0.406330,-0.304920,-0.172330,0.129580
3,Gemcitabine+17AAG+2,0,-0.38042,-0.476850,-0.28620,-0.309960,-0.789200,-0.52138,-0.23138,-0.19159,...,0.085960,-0.058587,-0.168620,-0.115580,0.122920,-0.058587,-0.396790,-0.327410,-0.189600,0.199510
4,Gemcitabine+17AAG+3,0,-0.34250,-0.456370,-0.31210,-0.284460,-0.748570,-0.50344,-0.24832,-0.20947,...,0.112590,-0.078223,-0.164030,0.132240,0.134300,-0.078223,-0.383420,-0.301940,-0.188050,0.192040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,DMSO+DMSO+4,0,-0.20042,0.190200,0.25483,-0.094821,0.176710,0.26651,0.23563,0.23331,...,0.140380,-0.028618,-0.115740,0.129690,0.139870,-0.028618,-0.112570,-0.140390,0.017725,0.113770
1532,DMSO+DMSO+5,0,-0.19125,0.190720,0.26269,-0.092610,0.080764,0.26569,0.25572,0.25968,...,0.230850,-0.034358,-0.232820,0.255570,0.258710,-0.034358,-0.224920,-0.274920,0.036213,0.208070
1533,DMSO+DMSO+5,0,-0.42625,0.252100,0.29306,-0.062056,0.148810,0.28713,0.26662,0.27103,...,0.229600,0.060381,-0.231070,0.254880,0.225020,0.060381,-0.239880,-0.309710,-0.067404,0.227050
1534,DMSO+DMSO+6,0,0.19065,0.036852,0.10324,-0.077310,0.058329,0.12246,0.08209,0.07820,...,0.027212,-0.019235,-0.067514,0.040317,-0.039779,-0.019235,-0.072545,-0.083441,0.015604,0.077095


In [302]:
#set training data and concentration
#Dna ： etoposide，Gemcitabine
#mt：paclitaxel，Vinblastine
#hsp：17aag，auy922

#prot："MG132"，ps341
#hdac：saha，apicidin
#这个函数要自动会设定label的
def set_training_label(single_df,training_label,name):
    for i in range(len(single_df)):
        training_label.append(name)



def extract_any_drug_set(frames1,concentration,drug_name1,drug_name2):
    if drug_name1!=drug_name2:
        for i in range(len(concentration)):
            frames1.append(df.loc[(df['drug_name'] =='%s+%s+'%(drug_name1,drug_name2)+str(concentration[i])) ])
        for i in range(len(concentration)):
            frames1.append(df.loc[(df['drug_name'] =='%s+%s+'%(drug_name2,drug_name1)+str(concentration[i])) ])
    elif drug_name1==drug_name2:
        #print(len(frames1))
        for i in range(len(concentration)):
            frames1.append(df.loc[(df['drug_name'] =='%s+%s+'%(drug_name1,drug_name2)+str(concentration[i])) ])
        #print(len(frames1))
        for i in range(len(concentration)):
            frames1.append(df.loc[(df['drug_name'] =='%s+DMSO+'%(drug_name1)+str(concentration[i])) ])
        for i in range(len(concentration)):
            frames1.append(df.loc[(df['drug_name'] =='DMSO+%s+'%(drug_name1)+str(concentration[i])) ])

#five_drug_list

five_drug=[]
#for i in range(11):
#    five_drug.append('f%s'%(i+1))

import copy
training_label=[]
def set_training_datasetxy(namelist,concentration,label):
    
    
    for i in range(len(namelist)):
        global frames
        frames=[]
        #print(len(frames))
        extract_any_drug_set(frames,concentration,namelist[i],namelist[i])
        global Dataframe
        Dataframe=pd.concat(frames)
        five_drug.append(Dataframe)
        #exec("%s=copy.deepcopy(Dataframe)"%five_drug[i],globals())

        set_training_label(Dataframe,label,namelist[i])
    


    
    
#effe_result=pd.concat(tem_five_drug_list)
#training_X=pd.concat([f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11])
set_training_datasetxy(['Etoposide','Gemcitabine','Paclitaxel',"Vinblastine","17AAG","AUY922","MG132","PS341","SAHA","Apicidin"],[1,2],training_label)

    

In [303]:
training_X=pd.concat(five_drug)
training_X_for_debug=copy.deepcopy(training_X)

In [304]:
training_X_for_debug

,drug_name,default,ks1,ks2,ks3,ks4,ks5,ks6,ks7,ks8,...,ks229,ks230,ks231,ks232,ks233,ks234,ks235,ks236,ks237,ks238
1260,Etoposide+Etoposide+1,0,-0.87250,0.430230,0.261280,0.266420,0.26622,0.239780,0.233630,0.220140,...,0.637980,0.104290,-0.218610,0.66386,0.662250,0.104290,-0.49506,-0.479380,-0.079277,0.201210
1261,Etoposide+Etoposide+1,0,-0.64125,0.469260,0.331120,0.251450,0.54936,0.363800,0.289990,0.292180,...,0.656250,-0.037638,0.073717,0.66479,0.706310,-0.037638,-0.41696,-0.328540,0.123200,-0.073301
1284,Etoposide+Etoposide+1,0,-0.92292,0.321250,0.202080,0.192920,0.43083,0.194580,0.176250,0.173750,...,0.570420,0.165830,-0.291670,0.66042,0.661670,0.165830,-0.52292,-0.476250,-0.128330,0.250000
1285,Etoposide+Etoposide+1,0,-0.79250,0.478660,0.342870,0.243060,0.49101,0.361770,0.306620,0.321320,...,0.616350,0.060051,-0.144120,0.62121,0.665220,0.060051,-0.47982,-0.412700,-0.069073,0.094167
1262,Etoposide+Etoposide+2,0,0.13685,0.093075,0.069296,-0.054434,0.16414,0.081740,0.044532,-0.048380,...,0.084730,-0.028454,0.078405,0.10014,0.094098,-0.028454,-0.09596,0.094325,0.088627,-0.079684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,Apicidin+DMSO+1,0,-0.56125,-0.127980,-0.202720,0.132560,-0.43827,-0.263750,-0.177440,-0.171990,...,-0.327150,-0.082372,0.204650,-0.34003,-0.328400,-0.082372,0.28657,0.243750,-0.139330,-0.165670
1106,Apicidin+DMSO+2,0,0.28421,-0.150070,-0.250800,0.295560,-0.24851,-0.335590,-0.224900,-0.196370,...,-0.076221,-0.057738,0.238720,-0.11695,-0.094161,-0.057738,0.15536,0.263890,0.056778,-0.240530
1107,Apicidin+DMSO+2,0,-0.28125,-0.027019,-0.112140,0.206540,-0.20146,-0.228520,-0.070881,-0.037207,...,0.092927,-0.063213,0.148810,0.10390,0.139170,-0.063213,0.08821,0.227660,0.040196,-0.165750
1130,Apicidin+DMSO+2,0,-0.49208,0.080930,-0.047459,0.174400,-0.12644,-0.162370,-0.037018,0.057045,...,0.162650,0.103890,0.103770,0.23528,0.255460,0.103890,-0.16771,0.168540,-0.091732,-0.103090


In [305]:
len(training_label)

120

In [306]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.preprocessing import StandardScaler
model = LinearDiscriminantAnalysis(n_components=2)
training_X=training_X.iloc[:,2:240].values
training_X_for_visualization=copy.deepcopy(training_X)
#visualization_data=visualization_data.iloc[:,2:240].values
training_X2=model.fit(training_X,training_label).transform(training_X)


In [307]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(training_X2, x=0, y=1, color=training_label)


fig.show()

# Combination

In [308]:
from itertools import combinations
single_drug=['Etoposide','Gemcitabine','Paclitaxel',"Vinblastine","17AAG","AUY922","MG132","PS341","SAHA","Apicidin","DMSO"] 
# Get all combinations of [1, 2, 3]
# and length 2
combi_name=[]
def combi(lst):
    comb = combinations(lst, 2)
    
    # Print the obtained combinations
    for i in list(comb):
        #print (i)
        if i[0]!='DMSO' and i[1]!="DMSO":
            combi_name.append([i[0],i[1]])
combi(single_drug)

In [309]:
combination_drug=[]
def set_visualization_datasetxy(namelist,concentration,label):
    for i in range(len(namelist)):
        global frames
        frames=[]
        extract_any_drug_set(frames,concentration,namelist[i][0],namelist[i][1])
        global Dataframe
        Dataframe=pd.concat(frames)
        combination_drug.append(Dataframe)
        if namelist[i][0]==namelist[i][1]:
            set_training_label(Dataframe,label,namelist[i][0])
        else:
            set_training_label(Dataframe,label,namelist[i][0]+'+'+namelist[i][1])
        

visualization_data_name=[]

for i in range(len(single_drug)):
    visualization_data_name.append([single_drug[i],single_drug[i]])
visualization_data_name+=combi_name


visualization_label=[]
set_visualization_datasetxy(visualization_data_name,[1,2,3,4,5,6],visualization_label)


In [327]:
len(visualization_data_name)

56

In [328]:
visualization_data_name

[['Etoposide', 'Etoposide'],
 ['Gemcitabine', 'Gemcitabine'],
 ['Paclitaxel', 'Paclitaxel'],
 ['Vinblastine', 'Vinblastine'],
 ['17AAG', '17AAG'],
 ['AUY922', 'AUY922'],
 ['MG132', 'MG132'],
 ['PS341', 'PS341'],
 ['SAHA', 'SAHA'],
 ['Apicidin', 'Apicidin'],
 ['DMSO', 'DMSO'],
 ['Etoposide', 'Gemcitabine'],
 ['Etoposide', 'Paclitaxel'],
 ['Etoposide', 'Vinblastine'],
 ['Etoposide', '17AAG'],
 ['Etoposide', 'AUY922'],
 ['Etoposide', 'MG132'],
 ['Etoposide', 'PS341'],
 ['Etoposide', 'SAHA'],
 ['Etoposide', 'Apicidin'],
 ['Gemcitabine', 'Paclitaxel'],
 ['Gemcitabine', 'Vinblastine'],
 ['Gemcitabine', '17AAG'],
 ['Gemcitabine', 'AUY922'],
 ['Gemcitabine', 'MG132'],
 ['Gemcitabine', 'PS341'],
 ['Gemcitabine', 'SAHA'],
 ['Gemcitabine', 'Apicidin'],
 ['Paclitaxel', 'Vinblastine'],
 ['Paclitaxel', '17AAG'],
 ['Paclitaxel', 'AUY922'],
 ['Paclitaxel', 'MG132'],
 ['Paclitaxel', 'PS341'],
 ['Paclitaxel', 'SAHA'],
 ['Paclitaxel', 'Apicidin'],
 ['Vinblastine', '17AAG'],
 ['Vinblastine', 'AUY922'],
 [

In [329]:
len(visualization_data)

1728

In [312]:
visualization_data=pd.concat(combination_drug)
visualization_help=copy.deepcopy(visualization_data)

In [313]:
visualization_help

,drug_name,default,ks1,ks2,ks3,ks4,ks5,ks6,ks7,ks8,...,ks229,ks230,ks231,ks232,ks233,ks234,ks235,ks236,ks237,ks238
1260,Etoposide+Etoposide+1,0,-0.87250,0.430230,0.261280,0.266420,0.266220,0.239780,0.233630,0.220140,...,0.63798,0.104290,-0.218610,0.66386,0.662250,0.104290,-0.495060,-0.479380,-0.079277,0.201210
1261,Etoposide+Etoposide+1,0,-0.64125,0.469260,0.331120,0.251450,0.549360,0.363800,0.289990,0.292180,...,0.65625,-0.037638,0.073717,0.66479,0.706310,-0.037638,-0.416960,-0.328540,0.123200,-0.073301
1284,Etoposide+Etoposide+1,0,-0.92292,0.321250,0.202080,0.192920,0.430830,0.194580,0.176250,0.173750,...,0.57042,0.165830,-0.291670,0.66042,0.661670,0.165830,-0.522920,-0.476250,-0.128330,0.250000
1285,Etoposide+Etoposide+1,0,-0.79250,0.478660,0.342870,0.243060,0.491010,0.361770,0.306620,0.321320,...,0.61635,0.060051,-0.144120,0.62121,0.665220,0.060051,-0.479820,-0.412700,-0.069073,0.094167
1262,Etoposide+Etoposide+2,0,0.13685,0.093075,0.069296,-0.054434,0.164140,0.081740,0.044532,-0.048380,...,0.08473,-0.028454,0.078405,0.10014,0.094098,-0.028454,-0.095960,0.094325,0.088627,-0.079684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,Apicidin+SAHA+5,0,0.37411,-0.065349,0.057443,0.022847,0.093927,0.068669,-0.049886,-0.054719,...,-0.14596,-0.077957,0.203860,-0.19735,-0.171840,-0.077957,0.174150,0.229100,0.110660,-0.201110
922,Apicidin+SAHA+6,0,0.21416,-0.432580,-0.468160,-0.186180,-0.314200,-0.433090,-0.470330,-0.481720,...,-0.13284,-0.060233,0.226730,-0.18511,-0.140490,-0.060233,-0.078018,0.181280,0.076847,-0.203830
923,Apicidin+SAHA+6,0,0.31547,-0.069626,0.042989,-0.028593,0.067919,0.053309,-0.052925,-0.060289,...,-0.16782,-0.057232,0.157560,-0.20424,-0.197800,-0.057232,0.218910,0.223010,0.091511,-0.164120
946,Apicidin+SAHA+6,0,0.38371,-0.162740,-0.220870,0.152090,-0.071784,-0.209780,-0.217170,-0.216290,...,-0.17512,-0.083596,0.211350,-0.23087,-0.221250,-0.083596,0.245400,0.280150,0.090551,-0.223120


# LDA model training

In [314]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.preprocessing import StandardScaler
model = LinearDiscriminantAnalysis(n_components=2)
#training_X=training_X.iloc[:,2:240].values
visualization_data=visualization_data.iloc[:,2:240].values
visualization_data=model.fit(training_X_for_visualization,training_label).transform(visualization_data)

In [315]:
#之前稀疏的画图都是没有train过，而且依旧238列

In [316]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(visualization_data, x=0, y=1, color=visualization_label)


fig.show()

# trace plotting

In [317]:
visualization_data

array([[ 2.1726132 , 11.552401  ],
       [ 1.30678793, 10.79531809],
       [ 1.50567317, 10.01643462],
       ...,
       [ 0.54390181,  6.77783973],
       [-0.60235359,  4.06030863],
       [-0.18028475,  4.54142154]])

In [318]:
visualization_label

['Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Etoposide',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcitabine',
 'Gemcit

In [319]:
combi_name[0]

['Etoposide', 'Gemcitabine']

In [320]:
visualization_help

,drug_name,default,ks1,ks2,ks3,ks4,ks5,ks6,ks7,ks8,...,ks229,ks230,ks231,ks232,ks233,ks234,ks235,ks236,ks237,ks238
1260,Etoposide+Etoposide+1,0,-0.87250,0.430230,0.261280,0.266420,0.266220,0.239780,0.233630,0.220140,...,0.63798,0.104290,-0.218610,0.66386,0.662250,0.104290,-0.495060,-0.479380,-0.079277,0.201210
1261,Etoposide+Etoposide+1,0,-0.64125,0.469260,0.331120,0.251450,0.549360,0.363800,0.289990,0.292180,...,0.65625,-0.037638,0.073717,0.66479,0.706310,-0.037638,-0.416960,-0.328540,0.123200,-0.073301
1284,Etoposide+Etoposide+1,0,-0.92292,0.321250,0.202080,0.192920,0.430830,0.194580,0.176250,0.173750,...,0.57042,0.165830,-0.291670,0.66042,0.661670,0.165830,-0.522920,-0.476250,-0.128330,0.250000
1285,Etoposide+Etoposide+1,0,-0.79250,0.478660,0.342870,0.243060,0.491010,0.361770,0.306620,0.321320,...,0.61635,0.060051,-0.144120,0.62121,0.665220,0.060051,-0.479820,-0.412700,-0.069073,0.094167
1262,Etoposide+Etoposide+2,0,0.13685,0.093075,0.069296,-0.054434,0.164140,0.081740,0.044532,-0.048380,...,0.08473,-0.028454,0.078405,0.10014,0.094098,-0.028454,-0.095960,0.094325,0.088627,-0.079684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,Apicidin+SAHA+5,0,0.37411,-0.065349,0.057443,0.022847,0.093927,0.068669,-0.049886,-0.054719,...,-0.14596,-0.077957,0.203860,-0.19735,-0.171840,-0.077957,0.174150,0.229100,0.110660,-0.201110
922,Apicidin+SAHA+6,0,0.21416,-0.432580,-0.468160,-0.186180,-0.314200,-0.433090,-0.470330,-0.481720,...,-0.13284,-0.060233,0.226730,-0.18511,-0.140490,-0.060233,-0.078018,0.181280,0.076847,-0.203830
923,Apicidin+SAHA+6,0,0.31547,-0.069626,0.042989,-0.028593,0.067919,0.053309,-0.052925,-0.060289,...,-0.16782,-0.057232,0.157560,-0.20424,-0.197800,-0.057232,0.218910,0.223010,0.091511,-0.164120
946,Apicidin+SAHA+6,0,0.38371,-0.162740,-0.220870,0.152090,-0.071784,-0.209780,-0.217170,-0.216290,...,-0.17512,-0.083596,0.211350,-0.23087,-0.221250,-0.083596,0.245400,0.280150,0.090551,-0.223120


In [321]:
import numpy as np
visualization_data=np.ndarray.tolist(visualization_data)

In [322]:
len(visualization_help)

1728

In [323]:
visualization_name_extract

['Etoposide+Etoposide+1',
 'Etoposide+Etoposide+1',
 'Etoposide+Etoposide+1',
 'Etoposide+Etoposide+1',
 'Etoposide+Etoposide+2',
 'Etoposide+Etoposide+2',
 'Etoposide+Etoposide+2',
 'Etoposide+Etoposide+2',
 'Etoposide+Etoposide+3',
 'Etoposide+Etoposide+3',
 'Etoposide+Etoposide+3',
 'Etoposide+Etoposide+3',
 'Etoposide+Etoposide+4',
 'Etoposide+Etoposide+4',
 'Etoposide+Etoposide+4',
 'Etoposide+Etoposide+4',
 'Etoposide+Etoposide+5',
 'Etoposide+Etoposide+5',
 'Etoposide+Etoposide+5',
 'Etoposide+Etoposide+5',
 'Etoposide+Etoposide+6',
 'Etoposide+Etoposide+6',
 'Etoposide+Etoposide+6',
 'Etoposide+Etoposide+6',
 'Etoposide+DMSO+1',
 'Etoposide+DMSO+1',
 'Etoposide+DMSO+1',
 'Etoposide+DMSO+1',
 'Etoposide+DMSO+2',
 'Etoposide+DMSO+2',
 'Etoposide+DMSO+2',
 'Etoposide+DMSO+2',
 'Etoposide+DMSO+3',
 'Etoposide+DMSO+3',
 'Etoposide+DMSO+3',
 'Etoposide+DMSO+3',
 'Etoposide+DMSO+4',
 'Etoposide+DMSO+4',
 'Etoposide+DMSO+4',
 'Etoposide+DMSO+4',
 'Etoposide+DMSO+5',
 'Etoposide+DMSO+5'

In [324]:
visualization_name_extract=[]
for each in visualization_help['drug_name']:
    visualization_name_extract.append(each)
    #print(each)
#for i in range(len(visualization_help)):
    #print(visualization_name_extract[i][:-2])

def combi_match(combi_name,i):
    global combi_string
    global single_string1
    global single_string2
    
    combi_string=combi_name[i][0]+'+'+combi_name[i][1]
    single_string1=combi_name[i][0]+'+'+combi_name[i][0]
    single_string2=combi_name[i][1]+'+'+combi_name[i][1]
#def extract_sets()
combi_match(combi_name,0)
combi_set=[]
single_set1=[]
single_set2=[]
for i in range(len(visualization_name_extract)):

    if single_string1==visualization_name_extract[i][:-2]:
        single_set1.append(visualization_data[i])
    elif single_string2==visualization_name_extract[i][:-2]:
        single_set2.append(visualization_data[i])
    elif combi_string==visualization_name_extract[i][:-2]:
        combi_set.append(visualization_data[i])
        

print(single_set1)

print(len(single_set1))
for i in range(0,13):
    exec("x%d"%i+"=[]")
    exec("y%d"%i+"=[]")
for i in range(0,24,4):
    x1.append(single_set1[i][0])
    y1.append(single_set1[i][1])
for i in range(1,24,4):
    x2.append(single_set1[i][0])
    y2.append(single_set1[i][1])
for i in range(2,24,4):
    x3.append(single_set1[i][0])
    y3.append(single_set1[i][1])
for i in range(3,24,4):
    x4.append(single_set1[i][0])
    y4.append(single_set1[i][1])


[[2.172613198455659, 11.55240100128826], [1.3067879262833935, 10.795318088465542], [1.5056731658055555, 10.016434621170273], [0.8168616962387666, 10.637521352828413], [-0.003820924208166161, 11.049426430841011], [0.08617630554995928, 9.585102010133793], [1.3001653063164282, 9.3808070411806], [1.0244266587745257, 11.661961737850715], [2.1323246150560844, 9.821317380385949], [-2.830054614569083, 8.479060582507255], [-2.5583078968466033, 9.220519540067034], [-0.49801322483709776, 9.739910730893063], [-2.127979399163828, 9.38289050910099], [3.52913382982862, 8.004433101174678], [2.8427992193637097, 12.765523889234888], [-1.0045542506598297, 10.518062025514437], [2.613990124949427, 12.73214668762711], [0.49185314143261355, 8.704967612684099], [-2.7725637502599656, 12.5433505234958], [5.472852181048045, 8.016989487651614], [3.3019787281921924, 11.330877683407268], [1.4949328005507905, 13.246518753351538], [2.0853300466431373, 8.53782907387011], [1.4924180855042795, 4.920001383440726]]
24


In [325]:
combi_string

'Etoposide+Gemcitabine'

In [326]:
#first connected dots
#black: combination
#blue:Etoposide
#purple: Paclitaxel
import plotly.io as pio
import kaleido

import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(visualization_data, x=0, y=1,opacity=0.2,color_discrete_sequence=['grey'],width=800, height=800) #color=grey_visualization_label)
#training['drug_name']
for i in range(2):
    exec('fig.add_trace(go.Scatter(x=x%d, y=y%d, mode="markers",line=dict(color="blue")))'%(i,i))

    exec('fig.add_trace(go.Scatter(x=x%d,y=y%d, mode="lines",line=dict(width=3, color="blue",dash="dot")))'%(i,i))
    
#for i in range(5,9):
#    exec('fig.add_trace(go.Scatter(x=x%d, y=y%d, mode="markers",line=dict(color="purple")))'%(i,i))
    
#    exec('fig.add_trace(go.Scatter(x=x%d,y=y%d, mode="lines",line=dict(width=3, color="purple",dash="dot")))'%(i,i))
    
#for i in range(9,13):
#    exec('fig.add_trace(go.Scatter(x=x%d, y=y%d, mode="markers",line=dict(color="black")))'%(i,i))
    
#    exec('fig.add_trace(go.Scatter(x=x%d,y=y%d, mode="lines",line=dict(width=3, color="black")))'%(i,i)) #row=1, col=1)
fig.add_annotation(dict(font=dict(color='brown',size=15),
                                        x=0,
                                        y=-0.12,
                                        showarrow=False,
                                        text="black:combination, blue: Etoposide, purple: Paclitaxel",
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
#fig.write_image("figure.png", engine="kaleido")
fig.show()
#fig.add_trace(go.Scatter(x=x2,y=y2, mode="lines",line=dict(width=2, color='skyblue')))
#trace2 = px.line(x=["a","b","c"], y=[50,70,90], title="second trace")

#fig.add_trace(trace2)
#fig.write_image("e+p.jpg", width=1920, height=1080)
#fig.write_image("e+p.png",engine='auto')

#plt.io.to_image(fig, format='png')
#image1=pio.to_image(fig,format='png')
fig.write_image("fig11.jpeg")
